In [1]:
#####training q_resnet18
import sys
import time
import random
import os
sys.path.append("..")
from module.layer_fusion_module import *
from module.resnet_module import *
from module.extract_weight_module import *
model_dir = "saved_models"
model_filename = "q_resnet18_cifar10.pt"
fusioned_model_filename = "q_fusioned_resnet18_cifar10.pt"
quantized_model_filename_jit = "q_resnet18_quantized_cifar10.pt"
quantized_model_filename = "q_resnet18_quantized_cifar10.pt"
model_filepath = os.path.join(model_dir, model_filename)
quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)
quantized_model_filepath_jit = os.path.join(model_dir, quantized_model_filename_jit)

def fuse_model(model,model_name):
    fusion_layer_dict= make_fuse_dict(model,model_name)
    print(fusion_layer_dict)
    for key in fusion_layer_dict:
        for fuse_group in fusion_layer_dict[key]:        
            torch.quantization.fuse_modules(eval(key), [fuse_group], inplace=True) 

num_classes = 10
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model = create_model(num_classes=num_classes)
model.to(cpu_device)

quantized_model = QuantizedResNet18(model_fp32=model)

quantization_config = torch.quantization.get_default_qconfig("fbgemm")

quantized_model.qconfig = quantization_config
fuse_model(quantized_model,'quantized_model')
torch.quantization.prepare_qat(quantized_model, inplace=True)
quantized_model.to(cpu_device)    
quantized_model = torch.quantization.convert(quantized_model, inplace=True)
quantized_model=load_model(model=quantized_model, model_filepath=quantized_model_filepath, device='cpu')
quantized_model.eval()


{'quantized_model.model_fp32.conv1': [['0', '1', '2']], 'quantized_model.model_fp32.conv2_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'quantized_model.model_fp32.conv2_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'quantized_model.model_fp32.conv3_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'quantized_model.model_fp32.conv3_x[0].shortcut': [['0', '1', '2']], 'quantized_model.model_fp32.conv3_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'quantized_model.model_fp32.conv4_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'quantized_model.model_fp32.conv4_x[0].shortcut': [['0', '1', '2']], 'quantized_model.model_fp32.conv4_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'quantized_model.model_fp32.conv5_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'quantized_model.model_fp32.conv5_x[0].shortcut': [['0', '1', '2']], 'quantized_model.model_fp32.conv5_x[1].residual_function': [['0', '1', '

C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:174: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."
C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:1109: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  Returning default scale and zero point "


QuantizedResNet18(
  (quant): Quantize(scale=tensor([0.0374]), zero_point=tensor([57]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (model_fp32): ResNet(
    (conv1): Sequential(
      (0): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.06772751361131668, zero_point=0, padding=(3, 3))
      (1): Identity()
      (2): Identity()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (conv2_x): Sequential(
      (0): q_BasicBlock(
        (residual_function): Sequential(
          (0): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.04158411920070648, zero_point=0, padding=(1, 1))
          (1): Identity()
          (2): Identity()
          (3): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.05617671459913254, zero_point=0, padding=(1, 1))
          (4): Identity()
          (5): Identity()
        )
        (shortcut): Sequential()
        (q_func): QFunctional(
          

In [3]:
###save zero_point
extract_model=quantized_model
original_zeropoint_list=list()
for parameter_name in quantized_model.state_dict():
    
    if parameter_name[-5:] == 'scale':

        eval_line='quantized_model.'
        c_past =''
        for c in parameter_name:
            if c_past =='.' and str.isdigit(c):
                eval_line=eval_line[:-1]+'['
                eval_line+=c
                eval_line+=']'
            else:
                eval_line+=c     
            c_past = c
        print(eval_line)
        print(quantized_model.state_dict()[parameter_name])
    elif  parameter_name[-10:] == 'zero_point':
        eval_line='quantized_model.'
        c_past =''
        for c in parameter_name:
            if c_past =='.' and str.isdigit(c):
                eval_line=eval_line[:-1]+'['
                eval_line+=c
                eval_line+=']'
            else:
                eval_line+=c     
            c_past = c
        print(eval_line)
        if( 'quant.' not in eval_line):
            exec("original_zeropoint_list.append("+eval_line+")")


quantized_model.quant.scale
tensor([0.0374])
quantized_model.quant.zero_point
quantized_model.model_fp32.conv1[0].scale
tensor(0.0883)
quantized_model.model_fp32.conv1[0].zero_point
quantized_model.model_fp32.conv2_x[0].residual_function[0].scale
tensor(0.0313)
quantized_model.model_fp32.conv2_x[0].residual_function[0].zero_point
quantized_model.model_fp32.conv2_x[0].residual_function[3].scale
tensor(0.0772)
quantized_model.model_fp32.conv2_x[0].residual_function[3].zero_point
quantized_model.model_fp32.conv2_x[1].residual_function[0].scale
tensor(0.0376)
quantized_model.model_fp32.conv2_x[1].residual_function[0].zero_point
quantized_model.model_fp32.conv2_x[1].residual_function[3].scale
tensor(0.1420)
quantized_model.model_fp32.conv2_x[1].residual_function[3].zero_point
quantized_model.model_fp32.conv3_x[0].residual_function[0].scale
tensor(0.0448)
quantized_model.model_fp32.conv3_x[0].residual_function[0].zero_point
quantized_model.model_fp32.conv3_x[0].residual_function[3].scale
ten

In [ ]:
###rcovery zero_point
count = 0
for parameter_name in quantized_model.state_dict():
    
    if parameter_name[-5:] == 'scale':

        eval_line='quantized_model.'
        c_past =''
        for c in parameter_name:
            if c_past =='.' and str.isdigit(c):
                eval_line=eval_line[:-1]+'['
                eval_line+=c
                eval_line+=']'
            else:
                eval_line+=c     
            c_past = c
        print(eval_line)
        print(quantized_model.state_dict()[parameter_name])
    elif  parameter_name[-10:] == 'zero_point':
        eval_line='quantized_model.'
        c_past =''
        for c in parameter_name:
            if c_past =='.' and str.isdigit(c):
                eval_line=eval_line[:-1]+'['
                eval_line+=c
                eval_line+=']'
            else:
                eval_line+=c     
            c_past = c
        print(eval_line)
        if( 'quant.' not in eval_line):
            exec(eval_line+" = original_zeropoint_list["+str(count)+"]")
            count+=1


In [3]:
#make zeropoint 0 except quant
train_loader, test_loader = prepare_dataloader(num_workers=0, train_batch_size=128, eval_batch_size=256)
_, int8_eval_accuracy = evaluate_model(model=quantized_model, test_loader=test_loader, device=cpu_device, criterion=None)
print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))    
for parameter_name in quantized_model.state_dict():
    
    if parameter_name[-5:] == 'scale':

        eval_line='quantized_model.'
        c_past =''
        for c in parameter_name:
            if c_past =='.' and str.isdigit(c):
                eval_line=eval_line[:-1]+'['
                eval_line+=c
                eval_line+=']'
            else:
                eval_line+=c     
            c_past = c
        print(eval_line)
        print(quantized_model.state_dict()[parameter_name])
    elif  parameter_name[-10:] == 'zero_point':
        eval_line='quantized_model.'
        c_past =''
        for c in parameter_name:
            if c_past =='.' and str.isdigit(c):
                eval_line=eval_line[:-1]+'['
                eval_line+=c
                eval_line+=']'
            else:
                eval_line+=c     
            c_past = c
        print(eval_line)
        if( 'quant.' not in eval_line):
            exec(eval_line+"=0")
            _, int8_eval_accuracy = evaluate_model(model=quantized_model, test_loader=test_loader, device=cpu_device, criterion=None)
            print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

Files already downloaded and verified
Files already downloaded and verified
INT8 evaluation accuracy: 0.682
quantized_model.quant.scale
tensor([0.0374])
quantized_model.quant.zero_point
quantized_model.model_fp32.conv1[0].scale
tensor(0.0677)
quantized_model.model_fp32.conv1[0].zero_point
INT8 evaluation accuracy: 0.682
quantized_model.model_fp32.conv2_x[0].residual_function[0].scale
tensor(0.0416)
quantized_model.model_fp32.conv2_x[0].residual_function[0].zero_point
INT8 evaluation accuracy: 0.682
quantized_model.model_fp32.conv2_x[0].residual_function[3].scale
tensor(0.0562)
quantized_model.model_fp32.conv2_x[0].residual_function[3].zero_point
INT8 evaluation accuracy: 0.682
quantized_model.model_fp32.conv2_x[0].q_func.scale
tensor(0.0890)
quantized_model.model_fp32.conv2_x[0].q_func.zero_point
INT8 evaluation accuracy: 0.682
quantized_model.model_fp32.conv2_x[1].residual_function[0].scale
tensor(0.0378)
quantized_model.model_fp32.conv2_x[1].residual_function[0].zero_point
INT8 evalu

In [ ]:
connected_scale_list=list()

connected_scale_list.append("quantized_model.model_fp32.conv1[0].scale")
connected_scale_list.append("quantized_model.model_fp32.conv2_x[0].residual_function[3].scale")
connected_scale_list.append("quantized_model.model_fp32.conv2_x[0].q_func.scale")
connected_scale_list.append("quantized_model.model_fp32.conv2_x[1].residual_function[3].scale")
connected_scale_list.append("quantized_model.model_fp32.conv2_x[1].q_func.scale")
connected_scale_list.append("quantized_model.model_fp32.conv3_x[0].residual_function[3].scale")
connected_scale_list.append("quantized_model.model_fp32.conv3_x[0].shortcut[0].scale")
connected_scale_list.append("quantized_model.model_fp32.conv3_x[0].q_func.scale")
connected_scale_list.append("quantized_model.model_fp32.conv3_x[1].residual_function[3].scale")
connected_scale_list.append("quantized_model.model_fp32.conv3_x[1].q_func.scale")
connected_scale_list.append("quantized_model.model_fp32.conv4_x[0].residual_function[3].scale")
connected_scale_list.append("quantized_model.model_fp32.conv4_x[0].shortcut[0].scale")
connected_scale_list.append("quantized_model.model_fp32.conv4_x[0].q_func.scale")
connected_scale_list.append("quantized_model.model_fp32.conv4_x[1].residual_function[3].scale")
connected_scale_list.append("quantized_model.model_fp32.conv4_x[1].q_func.scale")
connected_scale_list.append("quantized_model.model_fp32.conv5_x[0].residual_function[3].scale")
connected_scale_list.append("quantized_model.model_fp32.conv5_x[0].shortcut[0].scale")
connected_scale_list.append("quantized_model.model_fp32.conv5_x[0].q_func.scale")
connected_scale_list.append("quantized_model.model_fp32.conv5_x[1].residual_function[3].scale")
connected_scale_list.append("quantized_model.model_fp32.conv5_x[1].q_func.scale")

In [ ]:
#edit shortcut connected scale
for i in range(1,11):    
    unified_scale_value=i*0.01
    print("unified_scale_value:",unified_scale_value)
    for exec_line in connected_scale_list:
        exec(exec_line+"="+str(unified_scale_value))
        #print(exec_line)
    _, int8_eval_accuracy = evaluate_model(model=loaded_model, test_loader=test_loader, device=cpu_device, criterion=None)
    print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))